In [6]:
import subprocess
from pathlib import Path
import json
import sys
sys.path.append("..")
from scripts.client import VisAVisClient
from scripts.make_protocol import make_protocol

In [7]:
folder = '../movies/Channel_6x100/'
folder_path = Path(folder)
folder_path.mkdir(exist_ok=True, parents=True)

channel_height = 6
with open("../parameters.json", 'r') as json_file:
    PARAMETERS_DEFAULT = json.load(json_file)

In [8]:
client = VisAVisClient(
    visavis_bin=f'../target/bins/vis-a-vis-{channel_height}',
    sim_root=folder_path
)

protocol_file_path = make_protocol(
    pulse_intervals = [500],
    duration=2,
    out_folder='./'
)

In [9]:
def extinction_detector(states):
    df = states[states['seconds'] == 320]
    infected_sum = (df['E'] + df['I']).sum()
    return (infected_sum == 0)

def chaos_detector(states):
    df = states[states['seconds'] == 400]
    infected_sum = (df['E'] + df['I']).sum()
    return (infected_sum > 0)

            
def save_with_mp4(folder=folder):
    subprocess.run(f"rm {folder}/movie.mp4", shell=True)
    subprocess.run(f"ffmpeg -framerate 15 -pattern_type glob -i '{folder_path}/{folder}/*.png' -c:v libx264 {folder_path}/{folder}/movie.mp4", shell=True)


In [10]:
seed = 0
while True:
    seed += 1
    results =  client.run(
        parameters_json=PARAMETERS_DEFAULT,
        protocol_file_path = protocol_file_path,
        seed=seed
        )    
    if chaos_detector(results.states):
        dir_name = f"Simulation{seed}"
        results =  client.run(
            parameters_json=PARAMETERS_DEFAULT,
            protocol_file_path = protocol_file_path,
            seed=seed,
            dir_name=dir_name,
            images=True,
            clean_up=False
            ) 
        save_with_mp4(dir_name)
        break

Starting simulation ../movies/Test/sim_PYGGQJHBVWHJFEVI
Finished simulation ../movies/Test/sim_PYGGQJHBVWHJFEVI
Starting simulation ../movies/Test/sim_VIHFQWWMNZUAKVAN
Finished simulation ../movies/Test/sim_VIHFQWWMNZUAKVAN
Starting simulation ../movies/Test/sim_CGXCAZGFRTGEEIVL
Finished simulation ../movies/Test/sim_CGXCAZGFRTGEEIVL
Starting simulation ../movies/Test/sim_QUAHXTXXFECALABK
Finished simulation ../movies/Test/sim_QUAHXTXXFECALABK
Starting simulation ../movies/Test/sim_NZSMXANRGUIZDHLY
Finished simulation ../movies/Test/sim_NZSMXANRGUIZDHLY
Starting simulation ../movies/Test/Simulation5
Finished simulation ../movies/Test/Simulation5


rm: cannot remove 'Simulation5/movie.mp4': No such file or directory
ffmpeg version 6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 13 (GCC)
  configuration: --prefix=/usr --bindir=/usr/bin --datadir=/usr/share/ffmpeg --docdir=/usr/share/doc/ffmpeg --incdir=/usr/include/ffmpeg --libdir=/usr/lib64 --mandir=/usr/share/man --arch=x86_64 --optflags='-O2 -flto=auto -ffat-lto-objects -fexceptions -g -grecord-gcc-switches -pipe -Wall -Werror=format-security -Wp,-U_FORTIFY_SOURCE,-D_FORTIFY_SOURCE=3 -Wp,-D_GLIBCXX_ASSERTIONS -specs=/usr/lib/rpm/redhat/redhat-hardened-cc1 -fstack-protector-strong -specs=/usr/lib/rpm/redhat/redhat-annobin-cc1 -m64 -mtune=generic -fasynchronous-unwind-tables -fstack-clash-protection -fcf-protection -fno-omit-frame-pointer -mno-omit-leaf-frame-pointer' --extra-ldflags='-Wl,-z,relro -Wl,--as-needed -Wl,-z,now -specs=/usr/lib/rpm/redhat/redhat-hardened-ld -specs=/usr/lib/rpm/redhat/redhat-annobin-cc1 -Wl,--build-id=sha1 ' --extra-cflags=' -I/usr/i

In [11]:
subprocess.run("rm out.protocol", shell=True)

CompletedProcess(args='rm out.protocol', returncode=0)

## Seed

775098 - chaos for 24x60 channel

493523, 5, 99, 233 - chaos for 6x100 channed

34 - ext for 6x100 channel

let barrier_size: usize = 5;

for h in 0..barrier_size{
    // R barrier
    let ref mut c = lattice.cells[h*Lattice::WIDTH + w];
    c.molecules[Mol::I as usize] = 0;
    c.molecules[Mol::E as usize] = 0;
    c.molecules[Mol::R as usize] = 1;
}